# HW4: QA Agent

## Dependencies and LLM Backbone

In [1]:
# !pip install langchain==1.0.5
# !pip install langchain-core
# !pip install langchain-community
# !pip install faiss-cpu
# !pip install kagglehub

In [2]:
# This is the list of countries we are using (with their official languages)
# Feel free to use it in your code
list_of_countries = {}
with open("countries_with_languages.tsv", "r"  ) as f:
    for line in f.readlines():
        country, langs = line.strip().split("\t")
        list_of_countries[country] = langs.split(",")

### Choice 1: OpenAI API

The notebook's implementation is based on this.
Feel free to change the model, and please keep track of your usage on the "Usage" page on [LiteLLM API webpage](https://ai-gateway.andrew.cmu.edu/ui/).

In [3]:
# !pip install langchain-openai

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import getpass, os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
openai_model_id = "gpt-4o-mini-2024-07-18"
openai_embmodel_id = "azure/text-embedding-3-small"

llm = ChatOpenAI(
    model=openai_model_id,
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://ai-gateway.andrew.cmu.edu/"
)
embeddings = OpenAIEmbeddings(
    model=openai_embmodel_id,
    api_key=os.environ['OPENAI_API_KEY'],
    base_url='https://ai-gateway.andrew.cmu.edu/'
)

### Choice 2: Hugging Face Models

You may also use Hugging Face models without API credits if you have available GPU resource. You might have to the change prompt templates according to your model choice.

In [5]:
# !pip install langchain-huggingface text-generation transformers google-search-results 
# !pip install numexpr langchainhub sentencepiece sentence-transformers jinja2 bitsandbytes accelerate

In [6]:
import getpass, os
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")
hgf_model_id = "Qwen/Qwen3-0.6B"
hgf_embmodel_id = "sentence-transformers/all-mpnet-base-v2"

hgf_model = HuggingFacePipeline.from_model_id(
    model_id=hgf_model_id,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=128,
        do_sample=False,
    ),
)
hgf_llm = ChatHuggingFace(llm=hgf_model)
hgf_embeddings = HuggingFaceEmbeddings(model_name=hgf_embmodel_id)

2025-11-28 14:56:20.708736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764341780.725575   47552 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764341780.730872   47552 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-28 14:56:20.748247: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', '

## Handling different type of questions

Implement the answer formatting and extraction for each question type. You may change the prompt to fit your processing function.

In [7]:
from langchain.agents import create_agent

### 🗺️Global Trekker

In [8]:
global_trekker_messages = [
    {"role": "system", "content": "You are an expert in world knowledge."},
    {"role": "user", "content": "Given the following paragraph, guess the most likely country and city. Answer in the format of [country], [city]."},
]
global_trekker = create_agent(model=llm, tools=[])

In [9]:
def extract_global_trekker_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and city from the response
    # return country, city
    import re
    
    # Look for pattern [country], [city] or variations
    # Try to find content within brackets first
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        city = match.group(2).strip()
        return country, city
    
    # Fallback: look for two items separated by comma
    # Often LLMs will say something like "United States, Pittsburgh"
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                city = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Location:', 'Country:', 'City:']:
                    country = country.replace(prefix, '').strip()
                    city = city.replace(prefix, '').strip()
                return country, city
    
    # Last resort: return empty strings
    return "", ""

In [10]:
# Test run your extration function before using it in the main loop!
extract_global_trekker_answer("AAA, BBB")

('AAA', 'BBB')

### 🍽️Culinary Detective

In [11]:
import kagglehub
import pandas as pd

def gather_recipe_data(kaggledataset: str) -> list[str]:
    dataset_path = kagglehub.dataset_download(kaggledataset)
    df = pd.read_csv(f"{dataset_path}/Receipes from around the world.csv", encoding='latin-1')
    
    # Process the dataframe to list of text entries for retrieval
    # Format each recipe as structured text for better retrieval
    recipes = []
    for _, row in df.iterrows():
        # Create a readable text representation of each recipe
        recipe_parts = []
        for col in df.columns:
            value = row[col]
            # Skip NaN values and format nicely
            if pd.notna(value) and str(value).strip():
                recipe_parts.append(f"{col}: {value}")
        
        # Join all parts into a single text entry
        recipe_text = ". ".join(recipe_parts)
        recipes.append(recipe_text)
    
    return recipes

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool

recipes = gather_recipe_data("prajwaldongre/collection-of-recipes-around-the-world")
docs = [Document(page_content=recipe) for recipe in recipes]
vector = FAISS.from_documents(docs, embeddings)
retriever = vector.as_retriever(search_kwargs={"k": 2})

In [ ]:
@tool
def retrieve_recipes(query: str):
  """
  Retrieves recipes based on a search query.
  """
  return retriever.invoke(query)

culinary_detective_messages = [
    {"role": "system", "content": "You are an expert in world cuisines and culinary traditions."},
    {"role": "human", "content": "Given the following description, guess the most likely country and region. Answer in the format of [country], [region]."},
]
culinary_detective = create_agent(model=llm, tools=[retrieve_recipes])

In [ ]:
def extract_culinary_detective_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and region from the response
    # return country, region
    import re
    
    # Look for pattern [country], [region] or variations
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        region = match.group(2).strip()
        return country, region
    
    # Fallback: look for two items separated by comma
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                region = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Country:', 'Region:']:
                    country = country.replace(prefix, '').strip()
                    region = region.replace(prefix, '').strip()
                return country, region
    
    # Last resort: return empty strings
    return "", ""

### 👄Lingua Locale

In [ ]:
lingua_locale_messages = [
    {"role": "system", "content": "You are an expert in languages and cultural nuances."},
    {"role": "user", "content": "Given the following sentence, guess the which country's website it is from. Answer in the format of [country]."},
]
lingua_locale = create_agent(model=llm, tools=[])

In [ ]:
def extract_lingua_locale_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and "none" from the response
    # only the first field is used, the second is a dummy field to make the return type consistent
    # return country, "none"
    import re
    
    # Look for pattern [country] in brackets
    bracket_pattern = r'\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        return country, "none"
    
    # Fallback: look for country name in response
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Clean the line
        clean_line = line.strip().strip('[]"\'')
        # Remove common prefixes
        for prefix in ['The answer is', 'Answer:', 'Country:', 'The country is', 'This is from']:
            clean_line = clean_line.replace(prefix, '').strip()
        
        # If we have a short, cleaned line, it's likely the country
        if clean_line and len(clean_line) < 50:
            # Remove any trailing punctuation
            clean_line = clean_line.rstrip('.,;:')
            return clean_line, "none"
    
    # Last resort: return empty string
    return "", "none"

## Answering questions
This part includes how we load the questions and generate the prediction in desired format. 

In [ ]:
def geoguesser(q: dict, print_raw_response=False) -> tuple[str, str]:
    if q["type"] == "GlobalTrekker":
        query = {"role": "user", "content": f"Paragraph: {q['paragraph']}"}
        messages, agent, extractor = global_trekker_messages, global_trekker, extract_global_trekker_answer
    elif q["type"] == "CulinaryDetective":
        query = {"role": "user", "content": f"Ingredients: {q['ingredient']}. Description: {q['description']}"}
        messages, agent, extractor = culinary_detective_messages, culinary_detective, extract_culinary_detective_answer
    else: #q["type"] == "LinguaLocale":
        query = {"role": "user", "content": f"Sentence: {q['sentence']}"}
        messages, agent, extractor = lingua_locale_messages, lingua_locale, extract_lingua_locale_answer

    response_all = agent.invoke({"messages": messages + [query]})
    response = response_all["messages"][-1].content
    if print_raw_response: print(f"{q['type']}: {response_all}")
    return extractor(response)

In [ ]:
import json

# Here, we load the examples questions. Public/private set will be in the same format
dataset_name = "example.jsonl"
questions = []
with open(dataset_name, "r") as f:
    for line in f.readlines():
        questions.append(json.loads(line))

In [ ]:
# Test run on one question
# You might want to save the raw response for debugging answer formatting/extraction
# If the extracted answer seems off, check the raw response instead of running inference repeatedly
geoguesser(questions[0], print_raw_response=True)

GlobalTrekker: {'messages': [SystemMessage(content='You are an expert in world knowledge.', additional_kwargs={}, response_metadata={}, id='c0c4d44e-ea20-4aa5-8188-7c6839dafe8a'), HumanMessage(content='Given the following paragraph, guess the most likely country and city. Answer in the format of [country], [city].', additional_kwargs={}, response_metadata={}, id='d3a771c9-8b71-4b88-9f86-52fc4c0a6082'), HumanMessage(content="Paragraph: Picklesburgh is an annual event that takes place every July, named as a #1 Specialty Food Festival in the country. It celebrates the city's long pickle-making history, and the signature view is a giant Heinz pickle balloon floating above the venue.", additional_kwargs={}, response_metadata={}, id='c9032bdf-d3e4-4b24-9018-f6718d0fc4e8'), AIMessage(content='[United States], [Pittsburgh]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 98, 'total_tokens': 108, 'completion_tokens_details': {'a

('United States', 'Pittsburgh')

In [ ]:
# Sample script to generate answers
from tqdm import tqdm
answers = []
for q in tqdm(questions):
    try:
        country, category = geoguesser(q)
        answers.append(f"{q['type']}\t{country}\t{category}")
    except Exception as e:
        print(f"Error processing question {q}: {e}")
        answers.append(f"{q['type']}\tUnknown\tUnknown")

with open("example_output.txt", "w") as f:
    for answer in answers:
        f.write(answer + "\n")

100%|██████████| 9/9 [00:09<00:00,  1.04s/it]


## Evaluation
This is how we calculate the scores on Gradescope (details subject to change, but the general logic will stay the same).

In [ ]:
def soft_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer in answer:
        score = len(expectedAnswer) / len(answer)
    return score

def exact_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer == answer:
        score = 1.0
    return score

In [ ]:
answers = []
for q in questions:
    answers.append((q["type"], q["country"], q.get("city", q.get("region", ""))))
with open("example_output.txt", "r") as f:
    preds = [line.split("\t") for line in f.readlines()]

scores = {"GlobalTrekker": [], "CulinaryDetective": [], "LinguaLocale": []}
for (q_type, exp_country, exp_place), (p_type, pred_country, pred_place) in zip(answers, preds):
    assert q_type == p_type
    country_score = soft_match(pred_country, exp_country)
    category_score = 0.0
    weights = [0.0, 0.0]
    if q_type == "GlobalTrekker":
        #  correct country -> 80%, correct country and city -> +20%
        weights = [0.8, 0.2]
        if country_score > 0:
            if exp_place == "None": category_score = 1.0
            else: category_score = soft_match(pred_place, exp_place)
    elif q_type == "CulinaryDetective":
        # correct country -> 60%, correct country and region -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            if exp_place == "None": category_score = 1.0
            else: category_score = exact_match(pred_place, exp_place)
    else: # LinguaLocale
        # correct country -> 60%, matched official language -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            category_score = 1.0
        else: # incorrect country. language match works only if pred_country is a clean answer
            exp_langs = list_of_countries.get(exp_country, [])
            pred_langs = list_of_countries.get(pred_country, [])
            if any(lang in exp_langs for lang in pred_langs):
                category_score = 1.0

    score = weights[0] * country_score + weights[1] * category_score
    scores[q_type].append(score)

for q_type, score_list in scores.items():
    avg_score = sum(score_list) / len(score_list)
    print(f"{q_type} Average Score: {avg_score:.4f}")

GlobalTrekker Average Score: 0.8606
CulinaryDetective Average Score: 0.6000
LinguaLocale Average Score: 0.8000
